Este archivo es para implementar algunos algoritmos de aprendizaje por refuerzo y aplicarlos utilizando Open AI gym.

Se van a probar los siguientes 2 entornos (environments), de acuerdo con los resultados presentados en el articulo de Diego:

Pendulum-v0 y Hopper-v2

Tareas (Puede cambiar):
1. Terminar el curso de David Silver. (Completo: 100%)
2. Implementar con table lookup. (Completo: 100%)
3. Implementar con DQN. (En progreso: Aun no funciona)
4. Implementar con Simglucose (Sin comenzar)
5. Implementar con el trabajo de Diego.

# Instalación

In [1]:
#Correr esta celda si se utiliza Google Colab con un ambiente remoto.
!pip install progressbar2
!pip install gym

# Librerías

In [1]:
import time, copy, random
import numpy as np
import pandas as pd

import progressbar
import gym

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.initializers import RandomNormal

%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Clases

In [2]:
class ExperienceReplay():
    def __init__(self, tamano_minimo = 1000, tamano_maximo = 10**5, tamano_batch = 32, column_names = ['St', 'At', 'St+1', 'Rt+1', 'done']):
        self.tamano_buffer = 0
        self.tamano_batch = tamano_batch
        self.tamano_minimo = tamano_minimo
        self.tamano_maximo = tamano_maximo
        self.columnas = column_names
        
        self.buffer = self.inicializarBuffer()
        self.buffer.columns = column_names
        #self.indices = range(self.tamano_maximo)
    
    def inicializarBuffer(self):
        aux = pd.DataFrame(np.zeros((10**4,5)), dtype = 'object')
        #aux = pd.DataFrame({'St': None, 'At': [np.array([0,0])],
        #                   'Rt+1': [np.array([0,0])], 'St+1': [np.array([0,0])],
        #                   'done': [np.array([0,0])]})
        
        buffer = []
        
        for i in range(int(self.tamano_maximo/10**4)):
            buffer.append(aux)
        
        return pd.concat(buffer)
    
    def agregarDato(self, x, a, r, xp, d):
        
        self.buffer.iloc[self.tamano_buffer]['St'] = x
        self.buffer.iloc[self.tamano_buffer]['At'] = a
        self.buffer.iloc[self.tamano_buffer]['Rt+1'] = r
        self.buffer.iloc[self.tamano_buffer]['St+1'] = xp
        self.buffer.iloc[self.tamano_buffer]['done'] = d
                       
        self.tamano_buffer +=1
                       
        if(self.tamano_buffer == self.tamano_maximo):
            self.tamano_buffer = 0
    
    def darMuestras(self):
        #indices = random.sample(self.indices[0:self.tamano_buffer], self.tamano_batch)
        indices = random.sample(range(self.tamano_buffer), self.tamano_batch)
        return self.buffer.iloc[indices]


# Table Lookup

Pendulum-v0<br /> 
Utilizando el algoritmo de Sarsa.<br />
Está como chambón pero funciona.

In [ ]:
def darIndice(pS):
    return int(round(pS, 1)*10+10)

def escogerAccion(pS1, pS2, pS3):
    num = np.random.random()
    
    if(num <= epsilon):
        ind = np.random.randint(0,41)
        return ind
    else:
        ind = np.where(tablaQ[pS1, pS2, pS3, :] == np.amax(tablaQ[pS1, pS2, pS3, :]))
        ind = ind[0]
        
        if(len(ind) > 1):
            return np.random.choice(ind)
        else:
            return ind
    
    return accion

In [ ]:
envName = 'Pendulum-v0'

g = 0.9
alpha = 0.5
epsilon = 0.1 #Entre 0 y 1
maxIter = 1500000
ventana = 100
epsilonIterLimit = maxIter*0.9

tablaQ = np.ones((21,21,17,41))*-16
rewards = np.zeros(int(maxIter/10))

env = gym.make(envName)
observation = env.reset()

s1 = darIndice(observation[0])
s2 = darIndice(observation[1])
s3 = int(round(observation[2], 0) + 8)
a = escogerAccion(s1, s2, s3)

suma = 0

for i in progressbar.progressbar(range(maxIter)):
    
    if(i > maxIter-2000):
        env.render()
    
    observation, reward, done, info = env.step([(a-20)/10.0])
    observation = np.reshape(observation, (3,))
    
    s1p = darIndice(observation[0])
    s2p = darIndice(observation[1])
    s3p = int(round(observation[2], 0) + 8)
    
    ap = escogerAccion(s1p, s2p, s3p)
    
    r = reward #Por ahora
    suma+=r
    
    if(i % ventana == 0):
        rewards[int(i/ventana)] = suma/ventana
        suma = 0
    
    tablaQ[s1, s2, s3, a] = tablaQ[s1, s2, s3, a] + alpha*(r + g*tablaQ[s1p, s2p, s3p, ap] - tablaQ[s1,s2,s3,a])
    
    s1 = s1p
    s2 = s2p
    s3 = s3p
    a = ap
    
    if(i > epsilonIterLimit):
        epsilon = 0
    
env.close()

In [ ]:
for i in progressbar.progressbar(range(2000)):
    env.render()
    
    observation, reward, done, info = env.step([(a-20)/10.0])
    observation = np.reshape(observation, (3,))
    
    s1p = darIndice(observation[0])
    s2p = darIndice(observation[1])
    s3p = int(round(observation[2], 0) + 8)
    
    ap = escogerAccion(s1p, s2p, s3p)
    
    r = reward #Por ahora
    suma+=r
    
    if(i % ventana == 0):
        rewards[int(i/ventana)] = suma/ventana
        suma = 0
    
    tablaQ[s1, s2, s3, a] = tablaQ[s1, s2, s3, a] + alpha*(r + g*tablaQ[s1p, s2p, s3p, ap] - tablaQ[s1,s2,s3,a])
    
    s1 = s1p
    s2 = s2p
    s3 = s3p
    a = ap
    
env.close()

# DQN

In [3]:
############################################################
envName = 'Pendulum-v0'
env = gym.make(envName) 

num_acciones = 41
dim_state = 3

#Creando redes neuronales
def crearRedes(n1 = 20, n2 = num_acciones, learning_rate = 0.01):
    init = RandomNormal(seed = 0)
    
    modelo = Sequential()
    
    modelo.add(Dense(n1, activation = 'relu', kernel_initializer = init, bias_initializer = init, input_dim = dim_state))
    modelo.add(Dense(n1, activation = 'relu', kernel_initializer = init, bias_initializer = init))
    modelo.add(Dense(n2, activation = 'linear', kernel_initializer = init, bias_initializer = init))
    
    sgd = optimizers.sgd(lr = learning_rate)
    
    modelo.compile(loss = 'mean_squared_error', optimizer = sgd, metrics = ['accuracy'])
    
    return modelo, copy.deepcopy(modelo)

def actualizarRedTarget(pRedPrincipal, pRedTarget):
    
    for i in range(len(pRedPrincipal.layers)):
        pesosPrincipal = pRedPrincipal.layers[i].get_weights()
        pesosTarget = pRedTarget.layers[i].get_weights()
        
        pesos = [tao*l1 + (1-tao)*l2 for l1, l2 in zip(pesosPrincipal, pesosTarget)]
        
        pRedTarget.layers[i].set_weights(pesos)

def escogerAccion(pModelo, pX):
    num = np.random.random()
    
    if(num <= epsilon):
        ind = np.random.randint(0, num_acciones)
        return ind
    else:
        ind = np.argmax(pModelo.predict(pX))
        return ind

/home/hero/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
np.random.seed(123)       # For reproducibility

buffer = ExperienceReplay() # Instance of the replay_buffer (to record experiences)

# Create an instance of the neural network
red, red_target = crearRedes(learning_rate = 0.05)

num_episodes = 500             # Number of training episodes
num_steps = 100                 # Number of steps per episode
epsilon = 1.0                   # Initial exploration probability
tao = 0.001                     # Soft-update learning rate
tau = tao
gamma = 0.99                    # Discount factor
batch_size = 32                 # Batch size

total_steps_log = []            # Empty list to log number of steps per episode 
epsilon_log = []                # Empty list to log exploration probabilities
muestras_log = []
rewards_acumulados = []

initial_time = time.time()      # For comparison purposes

for e in progressbar.progressbar(range(num_episodes)):   # Loop over episodes
    state = env.reset()         # Reset environment
    state[2] *= 1/8.0
    state = np.reshape(state, (1, -1))
    
    rewardAcumulado = 0
    
    for s in range(num_steps):  # Loop over steps
        if(np.random.random() <= epsilon): # epsilon-greedy policy                    
            action = np.random.randint(num_acciones)
        else:
            action = np.argmax(red.predict(state))
            
        next_state, reward, done, info = env.step([(action-20.0)/10.0])  # Execute selected action
        next_state[2] *= 1/8.0
        next_state = np.reshape(next_state, (1, -1))
        
        buffer.agregarDato(state, action, reward, next_state, done)
       
        state = next_state   # Update state
        
        sumaMuestras = 0
        
        if(buffer.tamano_buffer >= buffer.tamano_minimo):   # Neural network training
            tm = time.time()
            muestras = buffer.darMuestras() # Sample a batch of experiences
            sumaMuestras += time.time()-tm
            
            states = np.concatenate(muestras['St'].values)
            actions = muestras['At'].values
            rewards = muestras['Rt+1'].values
            next_states = np.concatenate(muestras['St+1'].values)
            dones = muestras['done'].values
            
            # Forward pass over the DQN
            Q_values = red.predict(states)
            
            # Forward pass over the target DQN
            next_Q_values = red_target.predict(next_states)
            targets = Q_values.copy()   # Set initial target values

            targets[np.arange(buffer.tamano_batch), list(actions)] = rewards + gamma* np.max(next_Q_values, axis = 1) * (1 - dones)
            
            red.train_on_batch(states, targets)
            
            actualizarRedTarget(red, red_target)
        
        rewardAcumulado += reward
        
        if(done): # Termination condition
            break    
    
    sumaMuestras *= 1/float(s)
    muestras_log.append(sumaMuestras)
    rewards_acumulados.append(rewardAcumulado)
    
    total_steps_log.append(s)     # Append total number of steps
    epsilon_log.append(epsilon)   # Append exploration probability 
    epsilon = 0.995 * epsilon     # Exploration decay

total_training_time = time.time() - initial_time # Total training time    

plt.figure(figsize=(10,4))

plt.subplot(121)
plt.plot(rewards_acumulados)
plt.xlabel('Reward')
plt.ylabel('Episode')

plt.subplot(122)
plt.plot(muestras_log)
plt.xlabel('Episode')
plt.ylabel('Sample time')

plt.figure(figsize=(10,4))
plt.plot(epsilon_log)
plt.xlabel('Episode')
plt.ylabel('Epsilon')

print('Total training time: ', np.round(total_training_time, 2), 'seconds')

100% (500 of 500) |######################| Elapsed Time: 0:05:16 Time:  0:05:16


NameError: name 'subplot' is not defined

<Figure size 720x288 with 0 Axes>

In [9]:
observation = env.reset()

steps_prueba = 1000

state = env.reset()
state[2] *= 1/8.0
state = np.reshape(state, (1, -1))

for s in range(steps_prueba):
    
    action = np.argmax(red.predict(state))
    state, reward, done, info = env.step([(action-10.0)/20.0])
    
    state[2] *= 1/8.0
    state = np.reshape(state, (1, -1))
    
    env.render()
    
env.close()

In [35]:
#Creando redes neuronales
def crearRedes(n1 = 20, n2 = 4, learning_rate = 0.01):
    init = RandomNormal(seed = 0)
    
    modelo = Sequential()
    
    modelo.add(Dense(n1, activation = 'relu', kernel_initializer = init, bias_initializer = init, input_dim = 3))
    modelo.add(Dense(n2, activation = 'linear', kernel_initializer = init, bias_initializer = init))
    
    sgd = optimizers.sgd(lr = learning_rate)
    
    modelo.compile(loss = 'mean_squared_error', optimizer = sgd, metrics = ['accuracy'])
    
    return modelo, copy.deepcopy(modelo)

def actualizarRedTarget(pRedPrincipal, pRedTarget):
    
    for i in range(len(pRedPrincipal.layers)):
        pesosPrincipal = pRedPrincipal.layers[i].get_weights()
        pesosTarget = pRedTarget.layers[i].get_weights()
        
        pesos = [tao*l1 + (1-tao)*l2 for l1, l2 in zip(pesosPrincipal, pesosTarget)]
        
        pRedTarget.layers[i].set_weights(pesos)

def escogerAccion(pModelo, pX):
    num = np.random.random()
    
    if(num <= epsilon):
        ind = np.random.randint(0,41)
        return ind
    else:
        ind = np.argmax(pModelo.predict(pX))
        return ind

(20, 41)

In [16]:
env = Grid_World()      

modeloPrincipal, modeloTarget = crearRedes(learning_rate = 0.01)

#Con los parametros default.
buffer = ExperienceReplay()

gamma = 0.99
alpha = 0.35
epsilon = 1#Entre 0 y 1
epsilonDecay = 0.99
tao = 0.001

episodes = 1000
stepsPrueba = 500
maxSteps = 10
contadores = np.zeros((1,41))

rewardsAcumulados = np.zeros(episodes)

observation = env.reset()

x = np.reshape(observation, (1,3))
x[0,2] = x[0,2]/8.0

a = escogerAccion(modeloPrincipal, x)

#Este es un for inicial donde se almacena la cantidad minima de datos requerida en el buffer
#antes de comenzar con el entrenamiento.
for i in progressbar.progressbar(range(buffer.tamano_minimo)):
    observation, reward, done, info = env.step([(a-20.0)/10.0])
    
    xp = np.reshape(observation, (1,3))
    xp[0,2] = xp[0,2]/8.0
    x = xp
    
    buffer.agregarDato(x, a, reward, xp)
    
    #epsilon es 1 asi que siempre va a escoger al azar.
    a = escogerAccion(modeloPrincipal, x)
    
for e in progressbar.progressbar(range(episodes)):
    rewardAcumulado = 0
    env = gym.make(envName)
    observation = env.reset()
    x = np.reshape(observation, (1,3))
    x[0,2] = x[0,2]/8.0
    a = escogerAccion(modeloPrincipal, x)
    
    for s in range(maxSteps):
        observation, reward, done, info = env.step([(a-20.0)/10.0])
        
        rewardAcumulado += reward
        
        xp = observation
        xp = np.reshape(xp, (1,3))
        xp[0,2] = xp[0,2]/8.0

        buffer.agregarDato(x, a, reward, xp)

        muestras = buffer.darMuestras()

        #Pasar esto a un metodo solito
        xEnt = np.empty((1,3))
        aEnt = np.empty(1)
        rEnt = np.empty(1)
        xpEnt = np.empty((1,3))

        for m in muestras:
            xEnt = np.append(xEnt, m.get('St'), axis = 0)
            aEnt = np.append(aEnt, [m.get('At')], axis = 0)
            rEnt = np.append(rEnt, [m.get('Rt+1')], axis = 0)
            xpEnt = np.append(xpEnt, m.get('St+1'), axis = 0)

        xEnt = np.delete(xEnt, 0, axis = 0)
        aEnt = np.delete(aEnt, 0, axis = 0)
        rEnt = np.delete(rEnt, 0, axis = 0)
        xpEnt = np.delete(xpEnt, 0, axis = 0)

        qValores = modeloPrincipal.predict(xEnt)

        qValoresTarget = modeloTarget.predict(xEnt)
        maxQvalores = np.amax(qValoresTarget, axis = 1)

        #Aqui hay que calcular el maximo
        targets = np.add(rEnt, gamma*maxQvalores)

        yEnt = qValores

        for j in range(buffer.tamano_batch):
            yEnt[j,int(aEnt[j])] = targets[j]

        modeloPrincipal.fit(xEnt, yEnt, verbose = 0)
        actualizarRedTarget(modeloPrincipal, modeloTarget)

        x = xp
        a = escogerAccion(modeloPrincipal, x)

    epsilon = epsilon*epsilonDecay
    rewardsAcumulados[e] = rewardAcumulado

env = gym.make(envName)
observation = env.reset()
x = np.reshape(observation, (1,3))
x[0,2] = x[0,2]/8.0
a = escogerAccion(modeloPrincipal, x)    
    
for s in range(stepsPrueba):
    
    observation, reward, done, info = env.step([(a-20.0)/10.0])
    
    env.render()
    
    xp = observation
    xp = np.reshape(xp, (1,3))
    xp[0,2] = xp[0,2]/8.0
    
    a = escogerAccion(modeloPrincipal, xp)
    
env.close()

In [3]:
0.99**1000

4.317124741065786e-05

In [159]:
aEnt.shape

(2,)

In [6]:
contadores

array([[ 618.,  266.,  253.,  333., 1241.,  345.,  308.,  830., 1303.,
         797.,  452.,  332.,  983.,  149.,  362.,  178., 3307.,  268.,
         438., 1041.,  542.,  413.,  586.,  630.,  830.,  494.,  925.,
        1385., 1835.,  428.,  368.,  713.,  669.,  751., 1593.,  508.,
         770., 1066.,  255., 1183.,  252.]])

In [21]:
0.9999**50000

0.006736262610603238

In [ ]:
import gym
env = gym.make('Pendulum-v0')
for i_episode in range(20):
    observation = env.reset()
    for t in range(10):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

In [ ]:
print(env.observation_space)
print(env.action_space)

In [ ]:
print(env.observation_space.high)
print(env.observation_space.low)

In [ ]:
np.random.random()

In [ ]:
np.random.randint(0,6)

In [ ]:
test = np.array([10,22,50,-1,50])

In [ ]:
test3 = np.where(test == np.amax(test))

In [ ]:
len(test3[0])

In [ ]:
rewards[10/10]

In [43]:
rewards[300]

0.0

In [58]:
buffer.tamano_buffer

2000